## 电商打折套路解析

### 1、从现有数据中，分析出“各个品牌都有多少商品参加了双十一活动？”

    要求：
      ① 计算得到：商品总数、品牌总数
      ② 双十一当天在售的商品占比情况（思考：是不是只有双十一当天在售的商品是“参与双十一活动的商品？”）
      ③ 未参与双十一当天活动的商品，在双十一之后的去向如何？
      ④ 真正参与双十一活动的品牌有哪些？其各个品牌参与双十一活动的商品数量分布是怎样的？
       * 用绘制柱状图表示
#### 提示：

    ① 数据的“id”字段为商品的实际唯一标识，“title”字段则为商品在网页上显示的名称
       * 仔细看数据可以发现，同一个id的title不一定一样（双十一前后）
    ② 数据的“店名”字段为品牌的唯一标识
    ③ 按照商品销售节奏分类，我们可以将商品分为7类
       A. 11.11前后及当天都在售 → 一直在售
       B. 11.11之后停止销售 → 双十一后停止销售
       C. 11.11开始销售并当天不停止 → 双十一当天上架并持续在售
       D. 11.11开始销售且当天停止 → 仅双十一当天有售
       E. 11.5 - 11.10 → 双十一前停止销售
       F. 仅11.11当天停止销售 → 仅双十一当天停止销售
       G. 11.12开始销售 → 双十一后上架
    ④ 未参与双十一当天活动的商品，可能有四种情况：
       con1 → 暂时下架（F）
       con2 → 重新上架（E中部分数据，数据中同一个id可能有不同title，“换个马甲重新上架”），字符串查找特定字符 dataframe.str.contains('预售')
       con3 → 预售（E中部分数据，预售商品的title中包含“预售”二字）
       con4 → 彻底下架（E中部分数据），可忽略
    ⑤ 真正参加活动的商品 = 双十一当天在售的商品 + 预售商品 （可以尝试结果去重）
       通过上述几个指标计算，研究出哪些是真正参与双十一活动的品牌，且其商品数量是多少
   
### 2、哪些商品真的在打折呢？
    要求：
        ① 针对每个商品，评估其打折的情况
        ② 针对在打折的商品，其折扣率是多少
           * 绘制折线图：x轴为折扣率，y轴为商品数量
        ③ 按照品牌分析，不同品牌的打折力度
           * 绘制浮动散点图，y坐标为品牌类型，x坐标为折扣力度
        提示：
        ① 打折情况评估方法：
           真打折：商品的价格在10天内有波动、双11价格为10天内最低价、不存在涨价现象
           不打折：商品价格无变化
        ② 针对每个商品做price字段的value值统计，查看价格是否有波动，可以先用pd.cut将date分为不同周期：'双十一前','双十一当天','双十一后'，得到period字段
           data[['id','price','date']].groupby(['id','price']).min()
           针对统计出来的结果，如果按照id和price分组仍只有一个唯一值，则说明价格未变，没打折；否则为打折
        ③ 折扣率 = 双十一当天价格 / 双十一之前价格
        ④ 作图过程中，清除掉折扣率大于95%的数据


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import warnings
# 忽略警告
warnings.filterwarnings("ignore")
plt.rcParams["font.sans-serif"]=["SimHei"] #设置字体
plt.rcParams["axes.unicode_minus"]=False #该语句解决图像中的“-”负号的乱码问题

#### 1.读取文件,查出基本信息

In [ ]:
df = pd.read_excel('双十一淘宝美妆数据.xlsx')
df.head()

In [ ]:
df.info()

#### 2. 计算商品总数、品牌总数

#### 3 .统计不同商品的销售开始日期、截止日期

In [ ]:
df['day'] = df['update_time'].dt.day

In [ ]:
df.index = df['update_time']

In [ ]:
df.head()

In [ ]:
# df在售商品占比
data1 = df[["id","title","店名","day"]]
data1.head()

In [ ]:
# 对商品进行分类
df = pd.read_excel('双十一淘宝美妆数据.xlsx')
# 创建新day列,保存产品销售日期
df['day'] = df.update_time.dt.day
df.index = df.update_time

'''
    1. 双11 商品占比
    
'''
# 分析各品牌双11的占比
df_1 = df[['id',"title",'day']].copy()

# 可以通过日期的最大值和最小值来判断售卖区间
min_max_day = df_1[['id','day']].groupby('id').agg(['min','max'])['day'].copy()

# 筛选出双11当天销售的商品

day_11_id = df[df['day'] == 11]['id'].unique()
#拼接dataFrame,id为参加双11的商品
day_11_data = pd.DataFrame({"id":day_11_id,"双11售卖":True})

# 将df_1和双11售卖的数据合并,对商品进行分组
merge_data = pd.merge(min_max_day,day_11_data,how="left",on='id')

# 双11未售卖的填充为False
merge_data.fillna(False,inplace=True)

# 计算11 产品占比
p_total = len(df['id'].unique())

print("双11商品售卖占比:",'{:.2%}'.format(len(day_11_data)/p_total))


'''
    1. 产品销售分类分类
    
'''
merge_data['type'] = "分类"
merge_data['type'][(merge_data['min']<11) & (merge_data['max']>11) & (merge_data['双11售卖']==True)] = 'A'

merge_data['type'][(merge_data['min']<11) & (merge_data['max']==11)] = 'B'

merge_data['type'][(merge_data['min']==11) & (merge_data['max']>11)] = 'C'

merge_data['type'][(merge_data['min']==11) & (merge_data['max']==11)] = 'D'

merge_data['type'][(merge_data['min']<11) & (merge_data['max']<11)] = 'E'

merge_data['type'][(merge_data['min']<11) & (merge_data['max']>11) & (merge_data['双11售卖']==False)] = 'F'

merge_data[merge_data['min']>11] = 'G'

merge_data['type'].value_counts().plot.pie(autopct='%.2f%%',title="")
plt.show()

In [ ]:
# 未参与双十一当天活动的商品，在双十一之后的去向如何？
#   con1 → 暂时下架（F）
#   con2 → 重新上架（E中部分数据，数据中同一个id可能有不同title，“换个马甲重新上架”）
#   con3 → 预售（E中部分数据，预售商品的title中包含“预售”二字），字符串查找特定字符 dataframe.str.contains('预售')
#   con4 → 彻底下架（E中部分数据），可忽略
'''
    3. 未参加商品11的商品去向如何
    
'''
id_not11 = merge_data[merge_data['双11售卖']==False]

print('双十一当天没参加活动的商品总数为{}个，占比为{:.2%}\n-------'.format(len(id_not11),len(id_not11)/len(merge_data)))
print('双十一当天没参加活动的商品销售节奏类别为：\n',id_not11['type'].value_counts().index.tolist())

# con1 → 暂时下架（F）
id_con1 = merge_data[merge_data['type'] == 'F']['id'].values

#   con2 → 重新上架（E中部分数据，数据中同一个id可能有不同title，“换个马甲重新上架”）
#  -- 取出原始数据
data_not11 = pd.merge(id_not11,df,how='left',on="id")
# 按照id和title分组（找到id和title一对多的情况）
#计算id出现的次数，如果出现次数大于1，则说明该商品是更改了title的
data_con2 = data_not11[['id','title',"day"]].groupby(by = ['id','title']).count()
# 将id作为列计算重复个数
id_count = data_con2.reset_index()['id'].value_counts()
# 判断id个数大于1个,并取出id
id_con2 = id_count[id_count>1].index

# con3 → 预售（E中部分数据，预售商品的title中包含“预售”二字
data_con3 = data_not11[data_not11['title'].str.contains("预售")]   # 筛选出title中含有“预售”二字的数据
id_con3 = data_con3['id'].value_counts().index  

print("未参与双十一当天活动的商品中：\n暂时下架商品的数量为%i个，重新上架商品的数据量为%i个，预售商品的数据量为%i个" 
      % (len(id_con1), len(id_con2), len(id_con3)))

- 真正参与双十一活动的品牌有哪些？其各个品牌参与双十一活动的商品数量分布是怎样的？
- 真正参加活动的商品 = 双十一当天在售的商品 + 预售商品 （相加后再去重，去掉预售且当天在售的商品）

<img src="images/20220523111608.png" style="width:500px">

In [ ]:
# 真正参加活动的应该是预售的+双11当天的,将双11的id和 预售产品的id合并
real_11_id = np.hstack((day_11_id,id_con3))

print('商品总数为%i个' % p_total)
print('真正参加活动的商品商品总数为{}个，占比为{:.2%}\n-------' .format(len(real_11_id),len(real_11_id)/p_total))

# 品牌总数为 
 # 筛选出真正参与活动中 当天在售的商品id对应源数据
day_11_id_data = pd.DataFrame({'id':day_11_id})

data_11_source = pd.merge(day_11_id_data,df,how='left',on="id")
data_11_source_bc = data_11_source.groupby('店名')['id'].count()


day_11_pre_sale = pd.DataFrame({'id':id_con3})

day_11_pre_sale_source = pd.merge(day_11_pre_sale,df,how='left',on="id")

day_11_pre_sale_bc = day_11_pre_sale_source.groupby('店名')['id'].count()

result2_data = pd.DataFrame({'当天参与活动商品数量':data_11_source_bc,
                            '预售商品数量':day_11_pre_sale_bc})


result2_data['11活动总数'] = result2_data['当天参与活动商品数量']+result2_data['预售商品数量']
# 排序
result2_data.sort_values(by = '11活动总数',inplace = True,ascending = False)                                   
result2_data.plot.bar()  

### 2、哪些商品真的在打折呢？

要求：

    ① 针对每个商品，评估其打折的情况
    ② 针对在打折的商品，其折扣率是多少
       * 绘制折线图：x轴为折扣率，y轴为商品数量
    ③ 按照品牌分析，不同品牌的打折力度
       * 绘制散点图，y坐标为品牌类型，x坐标为折扣力度
    提示：
    ① 打折情况评估方法：
       真打折：商品的价格在10天内有波动、双11价格为10天内最低价、不存在涨价现象
       不打折：商品价格无变化
    ② 针对每个商品做price字段的value值统计，查看价格是否有波动，可以先用pd.cut将date分为不同周期：'双十一前','双十一当天','双十一后'，得到period字段
       data[['id','price','date']].groupby(['id','price']).min()
       针对统计出来的结果，如果按照id和price分组仍只有一个唯一值，则说明价格未变，没打折；否则为打折
    ③ 折扣率 = 双十一当天价格 / 双十一之前价格
    ④ 作图过程中，清除掉折扣率大于95%的数据

In [ ]:
data2 = df[['id','title','店名','day','price']].copy()
# 默认是包含右侧,不包含做些数据
data2['period'] = pd.cut(data2['day'],[4,10,11,14],labels = ['双十一前','双十一当天','双十一后'])
#print(data2.head())
# 筛选数据

price = data2[['id','price','period']].groupby(['id','price']).min()

price.reset_index(inplace = True)
# 针对每个商品做price字段的value值统计，查看价格是否有波动

id_count = price['id'].value_counts()
id_type1 = id_count[id_count == 1].index
id_type2 = id_count[id_count != 1].index
# 筛选出“不打折”和“真打折”的商品id

n1 = len(id_type1)
n2 = len(id_type2)
print('真打折的商品数量约占比%.2f%%，不打折的商品数量约占比%.2f%%' % (n2/len(id_count)*100, n1/len(id_count)*100))

In [ ]:
data2

In [ ]:
data2[['id','price','period','店名']].groupby(['id','period']).agg({"店名":"min","price":'min'})

In [ ]:
# 针对在打折的商品，其折扣率是多少

result3_data1 = data2[['id','price','period','店名']].groupby(['id','period']).agg({"店名":"min","price":'min'})
result3_data1.reset_index(inplace = True)
# 筛选数据

result3_before11 = result3_data1[result3_data1['period'] == '双十一前']
result3_at11 = result3_data1[result3_data1['period'] == '双十一当天']
result3_data2 = pd.merge(result3_at11,result3_before11,on = 'id')
# 筛选出商品双十一当天及双十一之前的价格

result3_data2['zkl'] = result3_data2['price_x'] / result3_data2['price_y']
# 计算折扣率


result3_data2

In [ ]:
#提取出id和折扣率
plot_data = result3_data2[['id','zkl']].dropna()
# 将折扣率拆分为20个分组
plot_data['zkl_range'] = pd.cut(plot_data['zkl'],
                                bins = np.linspace(0,1,21))
plot_data
# 按照分组进行统计个数
zkl_range_data = plot_data.groupby('zkl_range').count()
zkl_range_data
# 绘制图形
zkl_range_data['zkl'].plot()

In [ ]:
# 删除 

bokeh_data['zkl_range'] = pd.cut(bokeh_data['zkl'],bins = np.linspace(0,1,21))
bokeh_data2 = bokeh_data.groupby('zkl_range').count().iloc[:-1] # 这里去掉折扣率在0.95-1之间的数据，该区间内数据zkl大部分为1，不打折
bokeh_data2['zkl_pre'] = bokeh_data2['zkl']/bokeh_data2['zkl'].sum()